In [1]:
import numpy as np
import cv2
import os
import pickle
import imutils
import time
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
from imutils.video import VideoStream

2024-07-07 04:44:36.158038: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-07 04:44:36.191440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 04:44:36.191487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 04:44:36.192512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 04:44:36.198071: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-07 04:44:36.198952: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
confidence_value = 0.50

net = cv2.dnn.readNetFromCaffe("./deploy.prototxt.txt", "./res10_300x300_ssd_iter_140000.caffemodel")

videos = os.listdir("./videos/")
for video in videos:
	if(video == "processados"):
		break
	infos = video.split(".")[0].split("_")
	nome = infos[0]
	tipo = infos[1]

	vs = cv2.VideoCapture("./videos/"+video)
	saved_path = "./dataset/"+tipo
	read = 0
	saved = 0

	# loop over frames from the video file stream
	while True:
		# grab the frame from the file
		(grabbed, frame) = vs.read()
		# if the frame was not grabbed, then we have reached the end
		# of the stream
		if read >= 100:
			break
		# increment the total number of frames read thus far
		read += 1
		
		# grab the frame dimensions and construct a blob from the frame
		(h, w) = frame.shape[:2]
		blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
			(300, 300), (104.0, 177.0, 123.0))
		# pass the blob through the network and obtain the detections and
		# predictions
		net.setInput(blob)
		detections = net.forward()
		# ensure at least one face was found
		if len(detections) > 0:
			# we're making the assumption that each image has only ONE
			# face, so find the bounding box with the largest probability
			i = np.argmax(detections[0, 0, :, 2])
			confidence = detections[0, 0, i, 2]
			# ensure that the detection with the largest probability also
			# means our minimum probability test (thus helping filter out
			# weak detections)
			if confidence > confidence_value:
				# compute the (x, y)-coordinates of the bounding box for
				# the face and extract the face ROI
				box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
				(startX, startY, endX, endY) = box.astype("int")
				face = frame[startY:endY, startX:endX]
				# write the frame to disk
				p = os.path.sep.join([saved_path,"{}_{}_{}.png".format(nome,tipo,saved)])
				cv2.imwrite(p, face)
				saved += 1
				print("[INFO] saved {} to disk".format(p))
	# do a bit of cleanup
	vs.release()
	cv2.destroyAllWindows()

[INFO] saved ./dataset/real/ivan_real_0.png to disk
[INFO] saved ./dataset/real/ivan_real_1.png to disk
[INFO] saved ./dataset/real/ivan_real_2.png to disk
[INFO] saved ./dataset/real/ivan_real_3.png to disk
[INFO] saved ./dataset/real/ivan_real_4.png to disk
[INFO] saved ./dataset/real/ivan_real_5.png to disk
[INFO] saved ./dataset/real/ivan_real_6.png to disk
[INFO] saved ./dataset/real/ivan_real_7.png to disk
[INFO] saved ./dataset/real/ivan_real_8.png to disk
[INFO] saved ./dataset/real/ivan_real_9.png to disk
[INFO] saved ./dataset/real/ivan_real_10.png to disk
[INFO] saved ./dataset/real/ivan_real_11.png to disk
[INFO] saved ./dataset/real/ivan_real_12.png to disk
[INFO] saved ./dataset/real/ivan_real_13.png to disk
[INFO] saved ./dataset/real/ivan_real_14.png to disk
[INFO] saved ./dataset/real/ivan_real_15.png to disk
[INFO] saved ./dataset/real/ivan_real_16.png to disk
[INFO] saved ./dataset/real/ivan_real_17.png to disk
[INFO] saved ./dataset/real/ivan_real_18.png to disk
[IN

In [3]:
def build(width, height, depth, classes):
	# Inicializa o modelo sequencial e a forma de entrada
	model = Sequential()
	inputShape = (height, width, depth)
	chanDim = -1
	
	# Ajusta a forma de entrada se o formato da imagem for "channels_first"
	if K.image_data_format() == "channels_first":
		inputShape = (depth, height, width)
		chanDim = 1
		
	# Primeira camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(16, (3, 3), padding="same", input_shape=inputShape))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Segunda camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(16, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Primeira camada de pooling e dropout
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	
	# Terceira camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Quarta camada de convolução com ativação ReLU e normalização em lote
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	
	# Segunda camada de pooling e dropout
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	
	# Camada de flatten para transformar os dados em um vetor 1D
	model.add(Flatten())
	
	# Camada densa com ativação ReLU e normalização em lote
	model.add(Dense(64))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	
	# Camada de saída com ativação softmax para classificação
	model.add(Dense(classes))
	model.add(Activation("softmax"))
	
	# Retorna o modelo construído
	return model

In [4]:
# Definindo o tamanho do batch e o número de épocas
BS = 8
EPOCHS = 50

print("[INFO] loading images...")

# Carrega as imagens do diretório "./dataset/"
print("[INFO] loading images...")
imagePaths = list(paths.list_images("./dataset/"))
data = []
labels = []

# Loop através dos caminhos das imagens
for imagePath in imagePaths:
	# Extrai o rótulo da imagem a partir do caminho do arquivo
	label = imagePath.split(os.path.sep)[-2]
	
	# Carrega a imagem, redimensiona para 32x32 pixels e adiciona aos dados
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (32, 32))
	data.append(image)
	
	# Adiciona o rótulo "Falso" ou "Real"
	labels.append(label)

# Converte os dados em um array numpy e normaliza os valores dos pixels para o intervalo [0, 1]
data = np.array(data, dtype="float") / 255.0

# Codifica os rótulos como inteiros e, em seguida, converte para vetores one-hot
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

# Divide os dados em conjuntos de treinamento e teste
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# Inicializa o gerador de dados para aumentar os dados de treinamento
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
                         height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

print("[INFO] compiling model...")

# Constrói o modelo usando a função definida anteriormente
model = build(width=32, height=32, depth=3, classes=len(le.classes_))

# Compila o modelo usando a função de perda "binary_crossentropy" e o otimizador "adam"
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

print("[INFO] training network for {} epochs...".format(EPOCHS))

# Treina a rede neural utilizando os dados de treinamento aumentados e valida utilizando os dados de teste
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS), validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS, epochs=EPOCHS)


[INFO] loading images...
[INFO] loading images...


[INFO] compiling model...
[INFO] training network for 50 epochs...
Epoch 1/50
75/75 [==============================] - 3s 20ms/step - loss: 0.3197 - accuracy: 0.8750 - val_loss: 0.9456 - val_accuracy: 0.4900
Epoch 2/50
75/75 [==============================] - 2s 21ms/step - loss: 0.1979 - accuracy: 0.9350 - val_loss: 2.0051 - val_accuracy: 0.4900
Epoch 3/50
75/75 [==============================] - 2s 20ms/step - loss: 0.1941 - accuracy: 0.9333 - val_loss: 1.3594 - val_accuracy: 0.4900
Epoch 4/50
75/75 [==============================] - 1s 19ms/step - loss: 0.1291 - accuracy: 0.9583 - val_loss: 1.5001 - val_accuracy: 0.4900
Epoch 5/50
 1/75 [..............................] - ETA: 2s - loss: 0.3232 - accuracy: 0.8750

KeyboardInterrupt: 

In [ ]:
model_teste = "liveness.keras"

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=le.classes_))
# save the network to disk
print("[INFO] serializing network to '{}'...".format(model_teste))
model.save(model_teste)
# save the label encoder to disk
f = open("le.pickle", "wb")
f.write(pickle.dumps(le))
f.close()
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

[INFO] evaluating network...
61/61 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

       Falso       1.00      1.00      1.00       259
        Real       1.00      1.00      1.00       229

    accuracy                           1.00       488
   macro avg       1.00      1.00      1.00       488
weighted avg       1.00      1.00      1.00       488

[INFO] serializing network to 'liveness.keras'...


In [ ]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
net = cv2.dnn.readNetFromCaffe("./deploy.prototxt.txt", "./res10_300x300_ssd_iter_140000.caffemodel")
# load the liveness detector model and label encoder from disk
print("[INFO] loading liveness detector...")
model = load_model("liveness.keras")
le = pickle.loads(open("le.pickle", "rb").read())
# initialize the video stream and allow the camera sensor to warmup
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 600 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=600)
	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
		(300, 300), (104.0, 177.0, 123.0))
	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()
		# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with the
		# prediction
		confidence = detections[0, 0, i, 2]
		# filter out weak detections
		if confidence > 0.50:
			# compute the (x, y)-coordinates of the bounding box for
			# the face and extract the face ROI
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			# ensure the detected bounding box does fall outside the
			# dimensions of the frame
			startX = max(0, startX)
			startY = max(0, startY)
			endX = min(w, endX)
			endY = min(h, endY)
			# extract the face ROI and then preproces it in the exact
			# same manner as our training data
			face = frame[startY:endY, startX:endX]
			face = cv2.resize(face, (32, 32))
			face = face.astype("float") / 255.0
			face = img_to_array(face)
			face = np.expand_dims(face, axis=0)
			# pass the face ROI through the trained liveness detector
			# model to determine if the face is "real" or "fake"
			preds = model.predict(face)[0]
			j = np.argmax(preds)
			label = le.classes_[j]
			# draw the label and bounding box on the frame
			if(label == "Real"):
				label = "{}: {:.4f}".format(label, preds[j])
				cv2.putText(frame, label, (startX, startY - 10),
					cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
				cv2.rectangle(frame, (startX, startY), (endX, endY),
					(255, 0, 0), 2)
			else:
				label = "{}: {:.4f}".format(label, preds[j])
				cv2.putText(frame, label, (startX, startY - 10),
					cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
				cv2.rectangle(frame, (startX, startY), (endX, endY),
					(0, 0, 255), 2)
				# show the output frame and wait for a key press
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] loading face detector...
[INFO] loading liveness detector...
[INFO] starting video stream...
1/1 [==============================] - 0s 26ms/step
